In [16]:
#from copy import deepcopy

#from pyspark.ml.feature import Tokenizer, RegexTokenizer, StopWordsRemover, NGram, HashingTF, IDF, CountVectorizer, OneHotEncoder, StringIndexer, Word2Vec
from pyspark.sql.functions import col, udf, length, avg, lit, concat, size, array
from pyspark.sql.types import IntegerType, StringType, ArrayType
from pyspark.mllib.classification import NaiveBayes#, NaiveBayesModel
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

from pyspark.sql import SparkSession, SQLContext

#import re
#from nltk.stem import SnowballStemmer
#from nltk.stem import WordNetLemmatizer
#import nltk
#nltk.download('wordnet')


In [17]:
try:
    sc.stop()
except:
    print("no sc to stop")

In [18]:
sc

AttributeError: 'NoneType' object has no attribute 'sc'

<SparkContext master=local[*] appName=PySparkShell>

In [19]:
conf = SparkConf().setAppName('clickbait').setMaster('local[1]')
sc = SparkContext.getOrCreate(conf = conf)

spark = SparkSession.builder.appName('clickbait_appname').config("spark.driver.memory", "9g").getOrCreate()
sc

<SparkContext master=local[1] appName=clickbait>

In [20]:
sc

<SparkContext master=local[1] appName=clickbait>

In [21]:
# ORIGINAL SCHEMA FROM PRREPROC SCRIPT
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, ArrayType, DoubleType#, List
from pyspark.ml.linalg import VectorUDT
schema_preproc=StructType([StructField('id',StringType(),True),
                           StructField('title',StringType(),True),
                           StructField('text',StringType(),True),
                           StructField('label',StringType(),True),
                           StructField('s',StringType(),True),
                           StructField('ss',ArrayType(StringType(),True),True),
                           StructField('sss',ArrayType(StringType(),True),True),
                           StructField('ssss',ArrayType(StringType(),True),True),
                           StructField('ssss_1gram',ArrayType(StringType(),False),True),
                           StructField('ssss_2gram',ArrayType(StringType(),False),True),
                           StructField('ssss_3gram',ArrayType(StringType(),False),True),
                           StructField('3gram_feats',VectorUDT(),True),
                           StructField('label_index',DoubleType(),False)])

#schema_preproc=StructType(list(StructField('id',StringType(),True),StructField('title',StringType(),True),StructField('text',StringType(),True),StructField('label',StringType(),True),StructField('s',StringType(),True),StructField('ss',ArrayType(StringType(),True),True),StructField('sss',ArrayType(StringType(),True),True),StructField('ssss',ArrayType(StringType(),True),True),StructField('ssss_1gram',ArrayType(StringType(),False),True),StructField('ssss_2gram',ArrayType(StringType(),False),True),StructField('ssss_3gram',ArrayType(StringType(),False),True),StructField('3gram_feats',VectorUDT(),True),StructField('label_index',DoubleType(),False)))
#schema_preproc=StructType(List(StructField('id',StringType,True),StructField('title',StringType,True),StructField('text',StringType,True),StructField('label',StringType,True),StructField('s',StringType,True),StructField('ss',ArrayType(StringType,True),True),StructField('sss',ArrayType(StringType,True),True),StructField('ssss',ArrayType(StringType,True),True),StructField('ssss_1gram',ArrayType(StringType,False),True),StructField('ssss_2gram',ArrayType(StringType,False),True),StructField('ssss_3gram',ArrayType(StringType,False),True),StructField('3gram_feats',VectorUDT,True),StructField('label_index',DoubleType,False)))

In [22]:
PREPROCESSED_DF_JSON_PATH="file:///home/hadoop/cb/pre_train.csv/part-00000-6363f372-7d12-4684-9f07-8039ff1e6236-c000.json"
rddjson = sc.textFile(PREPROCESSED_DF_JSON_PATH)
#t_df = sqlContext.read.json(rddjson, schema=schema_preproc)
t_df = sqlContext.read.json(rddjson, schema=schema_preproc)

In [23]:
t_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- text: string (nullable = true)
 |-- label: string (nullable = true)
 |-- s: string (nullable = true)
 |-- ss: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- sss: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- ssss: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- ssss_1gram: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- ssss_2gram: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- ssss_3gram: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- 3gram_feats: vector (nullable = true)
 |-- label_index: double (nullable = false)



In [24]:
t_df.select('3gram_feats').limit(1).show()

Py4JJavaError: An error occurred while calling o208.showString.
: java.lang.IllegalStateException: SparkContext has been shutdown
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2053)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2082)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2101)
	at org.apache.spark.sql.execution.SparkPlan.executeTake(SparkPlan.scala:365)
	at org.apache.spark.sql.execution.CollectLimitExec.executeCollect(limit.scala:38)
	at org.apache.spark.sql.Dataset.org$apache$spark$sql$Dataset$$collectFromPlan(Dataset.scala:3389)
	at org.apache.spark.sql.Dataset$$anonfun$head$1.apply(Dataset.scala:2550)
	at org.apache.spark.sql.Dataset$$anonfun$head$1.apply(Dataset.scala:2550)
	at org.apache.spark.sql.Dataset$$anonfun$52.apply(Dataset.scala:3370)
	at org.apache.spark.sql.execution.SQLExecution$$anonfun$withNewExecutionId$1.apply(SQLExecution.scala:80)
	at org.apache.spark.sql.execution.SQLExecution$.withSQLConfPropagated(SQLExecution.


  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1067, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused
/usr/local/spark/python/pyspark/context.py:446: RuntimeWarning: Unable to cleanly shutdown Spark JVM process. It is possible that the process has crashed, been killed or may also be in a zombie state.
  RuntimeWarning"file:///home/hadoop/cb/pre_train.csv/part-00000-9f8f537d-20bc-43ef-972c-3705a1bb91aa-c000.json"
"file:///home/hadoop/cb/pre_train.csv/part-00000-6363f372-7d12-4684-9f07-8039ff1e6236-c000.json"




In [14]:
train, test = t_df.randomSplit([.8,.2], seed = 999)

In [15]:
NB = NaiveBayes(modelType= "multinomial", labelCol="label_index", featuresCol ="3gram_feats")
#NB = NaiveBayes(modelType= "bernoulli", labelCol="label_index", featuresCol ="3gram_feats")
nb_model= NB.fit(train)
nb_predictions= nb_model.transform(test)


NameError: name 'NaiveBayes' is not defined

In [ ]:

evaluator = MulticlassClassificationEvaluator(labelCol= "label_index", predictionCol = "prediction", 
                                              metricName="accuracy")

nb_accuracy = evaluator.evaluate(nb_predictions)

print("Accuracy : ", nb_accuracy)

In [ ]:
t_df.schema

In [ ]:
sc